In [1]:
# Mount Drive
import os
from google.colab import drive
drive.mount('/content/drive')

# common library
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# tensor computation
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from keras.utils import plot_model
from keras.models import Model, load_model
from keras.applications import MobileNetV2

Mounted at /content/drive


In [ ]:
base_dir = '/content/drive/Shareddrives/CAPSTONE C23-PS061/Dataset Split'

print("Contents of base directory:")
print(os.listdir(base_dir))

print("\nContents of train directory:")
print(os.listdir(f'{base_dir}/train'))

print("\nContents of validation directory:")
print(os.listdir(f'{base_dir}/validation'))

Contents of base directory:
['train', 'validation']

Contents of train directory:
['Kardus', 'Kaca (White & Green)', 'Plastik', 'Kertas', 'Logam', 'Organik', 'Botol Plastik', 'Botol Kaca']

Contents of validation directory:
['Kardus', 'Kaca (White & Green)', 'Plastik', 'Kertas', 'Logam', 'Organik', 'Botol Plastik', 'Botol Kaca']


In [ ]:
# Import train_dir & validation_dir
train_dir = f'{base_dir}/train'
validation_dir = f'{base_dir}/validation'

# Print the number of files in each class for train and validation
print("Number of files in each class (train):")
for class_folder in os.listdir(train_dir):
    class_folder_path = os.path.join(train_dir, class_folder)
    num_files = len(os.listdir(class_folder_path))
    print(f"{class_folder}: {num_files} files")

print("\nNumber of files in each class (validation):")
for class_folder in os.listdir(validation_dir):
    class_folder_path = os.path.join(validation_dir, class_folder)
    num_files = len(os.listdir(class_folder_path))
    print(f"{class_folder}: {num_files} files")

Number of files in each class (train):
Kardus: 623 files
Kaca (White & Green): 560 files
Plastik: 691 files
Kertas: 751 files
Logam: 538 files
Organik: 696 files
Botol Plastik: 700 files
Botol Kaca: 698 files

Number of files in each class (validation):
Kardus: 268 files
Kaca (White & Green): 241 files
Plastik: 304 files
Kertas: 322 files
Logam: 231 files
Organik: 299 files
Botol Plastik: 300 files
Botol Kaca: 300 files


In [ ]:
import tensorflow as tf

# Menentukan dimensi gambar dan jumlah kelas
image_width = 300
image_height = 300
num_classes = 8

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Pra-pemrosesan dan augmentasi gambar menggunakan ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255,  # Skala ulang intensitas pixel menjadi rentang 0-1
                                   rotation_range=20,  # Rotasi acak gambar dalam rentang 0-20 derajat
                                   width_shift_range=0.2,  # Pergeseran horizontal acak pada gambar
                                   height_shift_range=0.2,  # Pergeseran vertikal acak pada gambar
                                   horizontal_flip=True,  # Flip horizontal acak pada gambar
                                   )  

validation_datagen = ImageDataGenerator(rescale=1./255)  # Skala ulang intensitas pixel gambar validasi

# Menggunakan ImageDataGenerator untuk memuat dan mempersiapkan data latihan
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(image_width, image_height),
    batch_size=32,
    class_mode='categorical')

# Menggunakan ImageDataGenerator untuk memuat dan mempersiapkan data validasi
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(image_width, image_height),
    batch_size=32,
    class_mode='categorical')

Found 5257 images belonging to 8 classes.
Found 2265 images belonging to 8 classes.


In [ ]:
pretrained = MobileNetV2(input_shape = (image_width, image_height, 3), include_top = False, weights = 'imagenet')
pretrained.trainable = False
layers = Flatten()(pretrained.output)
layers = Dense(64, activation = 'relu')(layers)
outputs = Dense(8, activation = 'softmax')(layers)
model = Model(inputs = [pretrained.input], outputs = [outputs])

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

9406464/9406464 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 300, 300, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 150, 150, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 150, 150, 32  128         ['Conv1[0][0]']                  
                               

In [ ]:
# Melatih model menggunakan data latihan dan validasi
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // train_generator.batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.n // validation_generator.batch_size)

Epoch 1/10
164/164 [==============================] - 1889s 11s/step - loss: 1.5102 - accuracy: 0.7099 - val_loss: 0.5799 - val_accuracy: 0.8263
Epoch 2/10
164/164 [==============================] - 150s 912ms/step - loss: 0.5746 - accuracy: 0.8255 - val_loss: 0.6496 - val_accuracy: 0.8473
Epoch 3/10
164/164 [==============================] - 152s 927ms/step - loss: 0.4903 - accuracy: 0.8536 - val_loss: 0.7058 - val_accuracy: 0.8442
Epoch 4/10
164/164 [==============================] - 146s 889ms/step - loss: 0.3851 - accuracy: 0.8833 - val_loss: 0.3736 - val_accuracy: 0.8777
Epoch 5/10
164/164 [==============================] - 139s 847ms/step - loss: 0.3393 - accuracy: 0.8932 - val_loss: 0.4467 - val_accuracy: 0.9094
Epoch 6/10
164/164 [==============================] - 149s 909ms/step - loss: 0.3184 - accuracy: 0.9043 - val_loss: 0.3615 - val_accuracy: 0.9112
Epoch 7/10
164/164 [==============================] - 140s 856ms/step - loss: 0.2639 - accuracy: 0.9196 - val_loss: 0.4333 - 

In [ ]:
import matplotlib.pyplot as plt

# Mengambil nilai akurasi dan loss dari objek history
accuracy = history.history['accuracy']
loss = history.history['loss']
val_accuracy = history.history['val_accuracy']
val_loss = history.history['val_loss']

# Menghitung jumlah epoch
epochs = range(1, len(accuracy) + 1)

# Plot akurasi training dan validation
plt.plot(epochs, accuracy, 'b', label='Training Accuracy')
plt.plot(epochs, val_accuracy, 'r', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot loss training
plt.plot(epochs, loss, 'b', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Menyimpan model UBAH JUDUL JANGAN LUPA : )
model.save('/content/drive/Shareddrives/CAPSTONE C23-PS061/Model/MobileNetV2.h5')